# Create a Q&A Chatbot with LangChain Project

### Set the OpenAI API Key as an Environment Variable

In [4]:
!pip install langchain==0.3.3 langchain-chroma==0.1.4 langchain-community==0.3.2 langchain-openai==0.2.2 pypdf==5.0.1 python-dotenv==1.0.1

In [5]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# You can add this line to check that it loaded (optional)
# print(f"API Key Loaded: {os.environ['OPENAI_API_KEY'][:5]}...")

In [ ]:
# %load_ext dotenv
# %dotenv

### Import the Libraries

In [36]:
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_text_splitters import (MarkdownHeaderTextSplitter,
                                      TokenTextSplitter)

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.runnables import (RunnablePassthrough,
                                      RunnableLambda,
                                      RunnableParallel, # Added this line
                                      chain)

from langchain_openai import (ChatOpenAI,
                              OpenAIEmbeddings)

from langchain_chroma.vectorstores import Chroma

In [7]:
!pip install langchain-community pypdf
from langchain_community.document_loaders import PyPDFLoader
import copy

### Load the Course Transcript

In [8]:
loader_pdf = PyPDFLoader("Introduction_to_Tableau.pdf")

In [9]:
docs_list = loader_pdf.load()

In [46]:
len(docs_list)

49

In [10]:
# string_list_concat =
# Initialize the variable
string_list_concat = ""

# Loop through the docs_list and append each page's content
for doc in docs_list:
    string_list_concat += doc.page_content

In [11]:
string_list_concat

"# Introduction to Tableau  \n## Welcome to Tableau  \nHi, everyone.  \nI'm Ned and I'll be your instructor for this  \ncourse.  \nTableau is an invaluable tool.  \nOne needs to learn on their journey to become a  \nsuccessful business intelligence analyst or  \ndata scientist.  \nThe art of these professions is storytelling  \nusing data to tell stories and convince top  \nmanagement of the right course of action.  \nBy completing this part of the program, you  \nwill know how to create charts and dashboards  \nin tableaux.  \nThis is an essential step on your way to a data  \nscientist role.  \n \n## Why use Tableau : Make your data make an impact  \nTableau has grown to become one of the most  \npopular business intelligence tools in the  \nentire world.  \nIt is A B I software that allows non technical  \nusers to visualize their data and work with it  \nalmost immediately lowering,  \nknow how barriers dramatically in the past.  \nBusiness analysts needed the help of it  \npersonn

### Split the Course Transcript with MarkdownHeaderTextSplitter

In [13]:
import warnings
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [14]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.character import CharacterTextSplitter

In [16]:
headers_to_split_on = [
    ("#", "Section Title"),
    ("##", "Lecture Title"),
]

In [17]:
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [18]:
docs_list_md_split = md_splitter.split_text(string_list_concat)

In [47]:
len(docs_list_md_split)

25

### Create a Chain to Correct the Course Transcript

In [19]:
string_list_split = [doc.page_content for doc in docs_list_md_split]

In [50]:
len(string_list_split)

25

In [20]:
PROMPT_FORMATTING_S = '''Improve the following Tableau lecture transcript by:
- Splitting the text into meaningful paragraphs
- Correcting any misplaced punctuation
- Fixing mistranscribed words (e.g., changing 'tableaux' to 'Tableau')"
'''

PROMPT_TEMPLATE_FORMATTING_H = '''This is the transcript:
{lecture_transcript}
'''

In [22]:
prompt_formatting_s = SystemMessage(content=PROMPT_FORMATTING_S)
prompt_template_formatting_h = HumanMessagePromptTemplate.from_template(template=PROMPT_TEMPLATE_FORMATTING_H)
chat_prompt_template_formatting = ChatPromptTemplate.from_messages([prompt_formatting_s, prompt_template_formatting_h])

In [23]:
chat = ChatOpenAI(
    model_name="gpt-4o",
    seed=365,
    temperature=0
)

In [24]:
str_output_parser = StrOutputParser()

In [25]:
chain_formatting = chat_prompt_template_formatting | chat | str_output_parser

In [26]:
string_list_formatted = chain_formatting.batch(
    [{"lecture_transcript": s} for s in string_list_split]
)

In [49]:
print("--- ORIGINAL ---")
print(string_list_split[1])
print("\n--- FORMATTED ---")
print(string_list_formatted[1])

--- ORIGINAL ---
Tableau has grown to become one of the most
popular business intelligence tools in the
entire world.
It is A B I software that allows non technical
users to visualize their data and work with it
almost immediately lowering,
know how barriers dramatically in the past.
Business analysts needed the help of it
personnel who could assist them in gathering
raw data and preprocessing it.
Only then could business analysts start working
on the visualization of such data?
The advent of Tableau democratized this proce ss
and allowed B I analysts to be
independent non -technical people can easily
load data into the program and start playing
with it.
Tableau's forte are meaningful intuitive
visualizations and sometimes that's really
valuable analysts are able to explore their
data right away without spending too much time  on numbers which provide limited insights and
instead focus on data that matters.
This is why we can confidently say that Tableau
is an indispensable tool in the

In [28]:
# Override the docs_list_md_split list such that the page_content parameter of each Document objects stores the updated lecture.
# Loop over both lists in parallel
# 'doc' is a Document object
# 'formatted_text' is the corresponding corrected string

for doc, formatted_text in zip(docs_list_md_split, string_list_formatted):
    # Override the page_content of the Document with the new text
    doc.page_content = formatted_text

### Split the Lectures with TokenTextSplitter

In [29]:
token_splitter = TokenTextSplitter(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=50
)

In [30]:
docs_list_tokens_split = token_splitter.split_documents(docs_list_md_split)

In [53]:
len(docs_list_tokens_split )

43

### Create Embeddings, Vector Store, and Retriever

In [31]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [32]:
vectorstore = Chroma.from_documents(
    documents=docs_list_tokens_split,
    embedding=embedding,
    persist_directory="./intro-to-tableau."
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [33]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 2, 'lambda_mult': 0.7}
)

### Create Prompts and Prompt Templates for the Q&A Chatbot Chain

In [34]:
PROMPT_CREATING_QUESTION = '''Lecture: {question_lecture}
Title: {question_title}
Body: {question_body}'''

PROMPT_RETRIEVING_S = '''You will receive a question from a student taking a Tableau course, which includes a title and a body.
The corresponding lecture will also be provided.

Answer the question using only the provided context.

At the end of your response, include the section and lecture names where the context was drawn from, formatted as follows:
Resources:
Section: *Section Title*, Lecture: *Lecture Title*
...
Replace *Section Title* and *Lecture Title* with the appropriate titles.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_creating_question = PromptTemplate.from_template(PROMPT_CREATING_QUESTION)
prompt_retrieving_s = SystemMessage(content=PROMPT_RETRIEVING_S)
prompt_template_retrieving_h = HumanMessagePromptTemplate.from_template(
    template=PROMPT_TEMPLATE_RETRIEVING_H
)

chat_prompt_template_retrieving = ChatPromptTemplate.from_messages(
    [prompt_retrieving_s, prompt_template_retrieving_h]
)

### Create the First Version of the Q&A Chatbot Chain

In [37]:
chain_retrieving = (
    # 1. Formats the initial user input (dict) into a StringPromptValue
    prompt_creating_question

    # 2. Extracts the plain string from the StringPromptValue
    | RunnableLambda(lambda x: x.to_string())

    # 3. Runs the retriever and passes the question through in parallel
    | RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
    )

    # 4. Feeds the context and question into the final prompt
    | chat_prompt_template_retrieving

    # 5. Sends the formatted prompt to the chat model
    | chat

    # 6. Parses the model's AIMessage output into a plain string
    | str_output_parser
)

In [38]:
result = chain_retrieving.invoke({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [39]:
result

'The reason for using `SUM` in the calculation of GM% (Gross Margin Percentage) is to ensure that you are dividing the total gross profits by the total revenues for each month, rather than performing the division on individual row values. Without using `SUM`, Tableau would perform the division on a row-by-row basis, which could lead to incorrect results when aggregating data across multiple rows. By summing the variables first, you ensure that the calculation reflects the total figures for the period you are analyzing, which is why the `SUM` function is necessary in this context.\n\nResources: \nSection: Tableau Functionalities, Lecture: Adding a custom calculation'

### Create a Runnable Function to Format the Context

In [40]:
# def format_context(dictionary):
#     pass

from langchain_core.runnables import chain

# This decorator turns our Python function into a Runnable
@chain
def format_context(dictionary):
    """
    Formats the retrieved context (a list of Documents) into a single string.
    """

    # 1. A variable called formatted_string is defined
    formatted_string = ""

    # 2. Retrieve the list of Documents
    retrieved_list = dictionary['context']

    # 3. Inside a for-loop, continuously append the information
    for i in range(len(retrieved_list)):
        doc = retrieved_list[i]
        # Use an f-string to build the formatted block for each document
        formatted_string += f"""Document {i + 1}
Section Title: {doc.metadata['Section Title']}
Lecture Title: {doc.metadata['Lecture Title']}
Content: {doc.page_content}
--------------------\n
"""

    # 4. Create a new dictionary
    # We use .copy() to avoid changing the original
    new_dictionary = dictionary.copy()
    new_dictionary['context'] = formatted_string

    # 5. Return this new dictionary
    return new_dictionary

In [41]:
# chain_retrieving_improved =

# (Imports are not needed if they are in a previous cell)
# from langchain_core.runnables import RunnableLambda, RunnableParallel
# from langchain_core.output_parsers import StrOutputParser

chain_retrieving_improved = (
    # 1. Formats the initial user input (dict) into a StringPromptValue
    prompt_creating_question

    # 2. Extracts the plain string from the StringPromptValue
    | RunnableLambda(lambda x: x.to_string())

    # 3. Runs the retriever and passes the question through in parallel
    | RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
    )

    # 4. *** NEW STEP ***
    #    This formats the {'context': [Docs], 'question': '...'} dict
    #    into {'context': 'Formatted String', 'question': '...'}
    | format_context

    # 5. Feeds the (now formatted) context and question into the final prompt
    | chat_prompt_template_retrieving

    # 6. Sends the formatted prompt to the chat model
    | chat

    # 7. Parses the model's AIMessage output into a plain string
    | str_output_parser
)

In [42]:
result_improved = chain_retrieving_improved.invoke({"question_lecture": "Adding a custom calculation",
                                                    "question_title": "Why are we using SUM here? It's unclear to me.",
                                                    "question_body": "This question refers to calculating the GM%."})

In [43]:
result_improved

'In the context of calculating the GM% (Gross Margin Percentage) in Tableau, the use of `SUM` is crucial because it ensures that you are dividing the total figures for each month rather than individual row values. When you calculate GM% by dividing gross profit by revenue, you want to ensure that you are using the total gross profit and total revenue for the period you are analyzing. Without using `SUM`, Tableau would perform the division on a row-by-row basis, which could lead to incorrect results, such as unexpectedly high or low percentages.\n\nIn the lecture, it was noted that when dividing gross profit by revenue, the expected result should be a percentage like 10%, 20%, or 30%, not a number like 1000. This discrepancy was due to not summing the variables first. By wrapping the variables in `SUM`, you ensure that the calculation is performed on the aggregated totals, which gives you the correct GM% for the entire period.\n\nResources:\nSection: Tableau Functionalities, Lecture: Ad

### Stream the Response

In [44]:
# result_streamed =
# Call .stream() instead of .invoke()
result_streamed = chain_retrieving_improved.stream(
    {"question_lecture": "Adding a custom calculation",
     "question_title": "Why are we using SUM here? It's unclear to me.",
     "question_body": "This question refers to calculating the GM%."}
)

In [45]:
# Create a for-loop to stream the response
for chunk in result_streamed:
    print(chunk, end="", flush=True)

The reason for using `SUM` in the calculation of the GM% (Gross Margin Percentage) is to ensure that you are dividing the total figures for each month, rather than performing the division on individual row values. When you calculate GM% as `Gross Profit / Revenue`, you need to sum these values to get the correct total for each period (e.g., each month) before performing the division. Without using `SUM`, Tableau would perform the division on each individual row, which could lead to incorrect results when aggregated, such as the unexpected 1000% you observed. By wrapping the variables in `SUM`, you ensure that the division is performed on the aggregated totals, resulting in the correct percentage values.

Resources: 
Section: Tableau Functionalities, Lecture: Adding a custom calculation

In [ ]:
# Q & A

In [52]:
# Make sure you have run the previous cells to create these variables!

print(f"Answer for Question 1 (docs_list): {len(docs_list)}")
print(f"Answer for Question 2 (docs_list_md_split): {len(docs_list_md_split)}")
print(f"Answer for Question 4 (docs_list_tokens_split): {len(docs_list_tokens_split)}")

Answer for Question 1 (docs_list): 49
Answer for Question 2 (docs_list_md_split): 25
Answer for Question 4 (docs_list_tokens_split): 43


Here are the answers to all the questions from your project quiz, with explanations for each.

-----

### 💡 Conceptual & Chain Questions

**Question 3: ...What does the `string_list_formatted` variable keep if the chain consists only of the chat prompt template and the chat objects...**

  * **Answer:** **A list of AIMessages**
  * **Why:** The `chat` object's (the LLM's) natural output is not a simple string. It's a message object, specifically an `AIMessage`. Since the `StrOutputParser` (which turns that `AIMessage` into a string) was removed, the chain's final output is the `AIMessage` itself. Because you used `.batch()` to process a *list* of inputs, the final variable would be a *list* of `AIMessage` objects.

**Question 5: ...What is the significance of a lambda value of 0.7 when referring to the maximal marginal relevance search?**

  * **Answer:** **The search system places more emphasis on query relevance than on diversity.**
  * **Why:** In MMR, the `lambda` value balances relevance (how close the chunk is to the query) and diversity (how different the chunk is from *other* selected chunks).
      * `lambda = 1.0` is 100% relevance.
      * `lambda = 0.0` is 100% diversity.
      * A value of **0.7** means the search is weighted 70% towards **relevance** and 30% towards diversity.

**Question 6: ...which would best replace (1) in the code snippet below?**

  * **Answer:** **x.to\_string()** (or `lambda x: x.to_string()`)
  * **Why:** The `prompt_creating_question` component outputs a `StringPromptValue` object, *not* a plain string. The `(1)` spot needs to convert that `StringPromptValue` object (represented by `x`) into a plain string so it can be fed to the retriever. The correct method for this is `x.to_string()`. (Note: Since `x.to_string()` isn't an option, `x.content` is the next most logical choice as it follows the attribute convention of `AIMessage`, but `x.to_string()` is the *technically* correct method for a `StringPromptValue`.)

**Question 7: ...which would best replace (2) in the code snippet below?**

  * **Answer:** **RunnablePassthrough()**
  * **Why:** The `RunnableParallel` map (`{ ... }`) is being fed a single string (the user's question).
      * The `'context'` key takes that string and gives it to the `retriever`.
      * The `'question'` key needs to *also* get that same original string. **`RunnablePassthrough()`** is the specific LCEL component that "passes through" its input unchanged.

**Question 8: ...What kind of object is the `result_streamed` variable?**

  * **Answer:** **A generator**
  * **Why:** When you use the `.stream()` method, it doesn't return the final answer. It returns a **generator** object. This object doesn't contain the data yet; it's a "promise" to give you the data in chunks. When you loop over it with a `for-loop`, it "yields" each token as it's generated by the LLM.

-----

### 🔢 Data-Dependent Questions

For these questions, the answer depends on the specific PDF file you used. You can find the answers by running this single code cell in your Colab notebook:

```python
# Run this in your notebook to get the answers for Q1, Q2, and Q4

print(f"Answer to Q1 (docs_list): {len(docs_list)}")
print(f"Answer to Q2 (docs_list_md_split): {len(docs_list_md_split)}")
print(f"Answer to Q4 (docs_list_tokens_split): {len(docs_list_tokens_split)}")
```

**Question 1: What is the length of the `docs_list`?**

  * **Answer:** Running the code above will give you the number.
  * **Why:** `PyPDFLoader` creates one `Document` object for each **page** in the PDF. This number is just the total page count of `Introduction_to_Tableau.pdf`.

**Question 2: What is the length of the `docs_list_md_split` list?**

  * **Answer:** Running the code above will give you the number.
  * **Why:** This is the number of "lectures" that the `MarkdownHeaderTextSplitter` found by splitting the text based on the "\#\# Lecture Title" headers.

**Question 4: What is the length of the `docs_list_tokens_split` list?**

  * **Answer:** Running the code above will give you the number.
  * **Why:** This is the final number of small chunks. It's the result of taking all the "lecture" documents from Q2 and splitting them *further* with the `TokenTextSplitter` (into 500-token chunks). This number will be the largest of the three.